<a href="https://colab.research.google.com/github/wfos3241/TextMining/blob/main/ex04_%EC%9D%8C%EC%95%85%EC%B6%94%EC%B2%9C(%ED%95%9C%EA%B8%80)_%EC%BD%98%ED%85%90%EC%B8%A0%EA%B8%B0%EB%B0%98%ED%95%84%ED%84%B0%EB%A7%81_%EB%B0%B0%ED%8F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 학습내용
- 추천시스템
- 콘텐츠 기반 필터링을 활용한 음악 추천 시스템 구현
  - 데이터 전처리
    - 결측치 제거, 특수문자 제거
    - 형태소 분리, 어간추출, 정규화, 불용어 처리 (kiwi)
  - 임베딩 (Doc2Vec)
    - 문서 태깅
    - Doc2Vec 학습
  - 유사도 계산  
    - 가사 기반 유사도 계산 (코사인 유사도)
    - 년도 기반 유사도 계산
    - 가수 기반 유사도 계산
    - 가사, 년도, 가수에 가중치 적용한 최종 유사도 계산
  - 음악 추천 기능 구현

# 추천 시스템이란?

- 사용자(USER)에게 관련된 아이템(item)을 추천해주는 것

```
추천 ex)
A와 B가 넷플릭스에 가입했다고 가정

A와 B의 선호도
- A : 한국 드라마/영화, 로맨스물
- B : 미국 드라마/영화, 액션물

A에게는 `사이코지만 괜찮아`를 추천하고,
B에게는 `워킹데드`를 추천해주면 괜찮을 것 같음
```

- 추천 시스템 사례
  - e-commerce: 쿠팡과 같은 온라인 쇼핑몰에서 고객의 구매 이력과 검색 이력을 바탕으로 제품을 추천
  - 스트리밍 서비스: 넷플릭스, 유튜브, 스포티파이 등에서 사용자의 시청/청취 이력을 바탕으로 영화, 동영상, 음악을 추천
  - 뉴스 포털: 사용자가 관심을 가질 만한 뉴스 기사를 추천

- 추천 시스템의 종류

<center>  
<img src="https://arome1004.cafe24.com/images/machine_learning/recommand01.png" width=60%>
</center>

1. 콘텐츠 기반 필터링 (Contents-Based Filtering)
2. 협업(협력) 필터링 (Collaborative Filtering)
    - 기억 기반 (Memorial Based) 또는 최근접 이웃 (Nearest Neighbor) 기반
        - 사용자 기반 (User Based)
        - 아이템 기반 (Item Based)
    - 모델 기반 협업 필터링
        - 잠재 요인 (Latent Factor) 협업 필터링 → 행렬 인수분해
        - ML & DL 포함
3. Hybrid 방식
  - 추천 시스템의 초창기 : "콘텐츠 기반"/"최근접 이웃 기반 협업" 필터링이 주로 사용
  - "넷플릭스 추천 시스템 경연 대회"에서 "잠재 요인 협업 필터링" 방식이 우승하면서, 많은 추천 시스템의 대중화
  - 서비스하는 아이템 특성에 따라 콘텐츠 기반이나 최근접 이웃 기반을 유지하는 사이트도 존재
  - 요즘에는 "개인화 특성을 좀 더 강화"하기 위한 "하이브리드 형식(콘텐츠와 협업을 적절히 결합)"이나 "딥러닝 기반"이 많이 활용

- 콘텐츠 기반 필터링 (Contents-Based Filtering)
  - 사용자가 특성 콘텐츠를 선호한다면 유사한 콘텐츠를 추천해주는 방식
  - 유사도 기반으로 쉽게 구현 가능
  - (예) 사용자가 A영화에 높은 평점을 주었는데 A영화는 액션 영화이고 홍길동이라는 감독이었다면 홍길동 감독이 만든 다른 액션 영화을 추천


<center>  
<img src="https://arome1004.cafe24.com/images/machine_learning/recommand02.png" width=30%>
</center>

# 가사(lyric)를 활용한 유사도 기반 추천 시스템 구현
- 콘텐츠 기반 필터링 기법 활용

- 데이터 로드

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/텍스트마이닝

/content/drive/MyDrive/Colab Notebooks/텍스트마이닝


- 데이터 로드
  - 컬럼 구성

| 컬럼명 | 설명 |
| --- | --- |
| id | 각 노래의 고유 식별자 |
| year | 노래가 발표된 연도 |
| title | 노래 제목 |
| singer | 노래 가수 이름 |
| lyric | 노래 가사 |
| x_rated | X-rated 등급 여부 |

In [ ]:
import pandas as pd

# 1964년부터 2023년까지 멜론의 연도별 Top 100 노래 리스트

data = pd.read_csv("./data/lyrics_by_year_1964_2023.csv")
data.head()

,id,year,title,singer,lyric,x_rated
0,30072384,1964,워싱턴광장,정 시스터즈,NaN,False
1,8150699,1964,황혼의 에레지,최양숙,NaN,False
2,5758967,1964,물새우는 해변,권혜경,고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 ...,False
3,4083218,1964,내일또 만납시다,금호동,하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\...,False
4,3622464,1964,밀짚모자 목장아가씨,박재란,시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술...,False


In [ ]:
# 결측치 확인
data.info()

# lyric 컬럼에 결측치가 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4666 entries, 0 to 4665
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       4666 non-null   int64 
 1   year     4666 non-null   int64 
 2   title    4666 non-null   object
 3   singer   4666 non-null   object
 4   lyric    4458 non-null   object
 5   x_rated  4666 non-null   bool  
dtypes: bool(1), int64(2), object(3)
memory usage: 187.0+ KB


### 전처리
- 결측치 제거 & 인덱스 재설정

In [ ]:
# drop = True : 제거한 인덱스를 컬럼으로 추가하지 않음

data.dropna(subset=["lyric"], inplace=True)
data.reset_index(drop=True, inplace=True)

- 특수문자 제거
  - 한글, 영문, 숫자, 빈공백, ?, !를 제외한 특수문자 삭제

In [ ]:
import re

# \s : 빈 공백과 \n도 포함

pattern = r"[^a-zA-Z0-9가-힣\s\?\!\.]"

# \n까지 삭제하는 패턴 (\s대신 빈공백으로 표시)

pattern2 = r"[^a-zA-Z0-9가-힣 \?\!\.]"

# data["lyric"] 컬럼의 데이터를 하나씩 가져와서 lyric 변수에 저장
# lyric의 데이터를 패턴에 맞는 데이터를 제거

cleaned_lyrics = [re.sub(pattern, "", lyric) for lyric in data["lyric"]]
cleaned_lyrics[:3]

['고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 물새의 울음소리\n쓸쓸한 내 마음 속에 슬픔을 주네\n고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 물새의 울음소리\n쓸쓸한 내 마음 속에 슬픔을 주네\n',
 '하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\n가로등 하나 둘 꽃 피네\n허공을 스치는 바람은 차고\n흐뭇한 마음은 애드베룬\n가벼운 발길 헤어질 때 인사는\n내일 또 다시 만납시다\n하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\n가로등 하나 둘 꽃 피네\n허공을 스치는 바람은 차고\n흐뭇한 마음은 애드베룬\n가벼운 발길 헤어질 때 인사는\n내일 또 다시 만납시다\n내일 또 다시 만납시다\n',
 '시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술에는\n살며시 웃음 띄우고\n넓다란 푸른 목장\n하늘에 구름가네\n라라라 라라라라라\n라라라라 라라라라\n라라라 라라라라라\n라라라라 라라라라\n연분홍 빛 입술에는\n살며시 웃음 띄우고\n넓다란 푸른 목장\n하늘에 구름가네\n구름가네 음음음음\n']

- cleaned_lyric 컬럼에 결과 저장

In [ ]:
data["cleaned_lyrics"] = cleaned_lyrics
data.head()

,id,year,title,singer,lyric,x_rated,cleaned_lyrics
0,5758967,1964,물새우는 해변,권혜경,고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 ...,False,고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 ...
1,4083218,1964,내일또 만납시다,금호동,하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\...,False,하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\...
2,3622464,1964,밀짚모자 목장아가씨,박재란,시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술...,False,시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술...
3,3621978,1964,아빠 안녕,현미,비가 나리네 비가 나리네\n돌아선 이가슴에\n그님은 어데 그어느곳에\n이마음 벗사려...,False,비가 나리네 비가 나리네\n돌아선 이가슴에\n그님은 어데 그어느곳에\n이마음 벗사려...
4,2517558,1964,빗속의 여인 1964년작,Add 4 신중현,잊지 못할 빗속의 여인\n지금은 어데 있나\n\n노오란 레인코트에\n검은 눈동자 잊...,False,잊지 못할 빗속의 여인\n지금은 어데 있나\n\n노오란 레인코트에\n검은 눈동자 잊...


### 형태소 분석 및 불용어 처리
- 불용어 사전 로드 및 형태소 분석기 초기화

In [ ]:
!pip install -q kiwipiepy konlpy

In [ ]:
from konlpy.tag import Okt
from kiwipiepy import Kiwi

In [ ]:
# 등록된 불용어 내용 확인

from kiwipiepy.utils import Stopwords

stopword = Stopwords()
okt = Okt()
kiwi = Kiwi()

stopwords_list = [word for word, tag in stopword.stopwords]

print(stopwords_list)

['는', 'ᆯ', '의', '과', '만', '다', '이', '중', '이', '지', '보', '이', '우리', '말', '어', '이', '에서', '과', 'ᆫ다', '주', '그', '라', '다고', '부터', '다는', '한', '로', '면서', '적', '를', '을', '받', '더', '며', '명', '어서', '때문', '그', '에게', '까지', '면', '하', '도', '되', '으로', '은', '있', '일', '만', '들', '나', '기', '지', '따르', '어야', '지만', '던', '통하', '되', '않', '있', '지역', '제', '사람', '다', '어', '고', '에', '게', '고', '화', '는', '년', '겠', '수', '은', '하', '때', '것', '을', '하', '등', '원', '가', '라는', '월', '었', '없', '와', 'ᆫ', '하', '이', '대하', '아니', '위하', '같', '와', '일', 'ᆫ', '성']


In [ ]:
# 형태소의 종류 출력

okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

- 형태소 분리, 불용어 처리, 어간 추출, 정규화 처리
  - 노래의 감성에 영향을 많이 줄 만한 아래 4가지 품사를 활용
  - 실험을 통한 분석 결과를 기반으로 품사 설정하는 것을 권장

In [ ]:
def pos_tagging(text) :

  # 띄어쓰기 교정

  text = kiwi.space(text)

  # 형태소 분리 (어간추출, 정규화)

  pos_word = okt.pos(text, norm = True, stem = True)

  # 원하는 품사만 선택 / 불용어 처리

  tagged_list = []

  # 형태소와 품사를 따로 저장

  for word, tag in pos_word :

    # 해당 품사라면

    if tag in ["Adjective", "Adverb", "Alpha", "Exclamation", "Noun", "Number", "Verb"] :

      # 불용어가 아니라면

      if word not in stopwords_list :
        tagged_list.append(word)

  return tagged_list

In [ ]:
from tqdm import tqdm

# data["cleaned_lyrics"] 컬럼의 값을 하나씩 가져와서 text에 저장
# text를 pos_tagging()로 전달
# pos_tagging()의 결과를 리스트에 저장

tagged_lyric = [pos_tagging(text) for text in tqdm(data["cleaned_lyrics"])]

100%|██████████| 4458/4458 [04:56<00:00, 15.02it/s]


- 결과를 tagged_lyric 컬럼에 추가

In [ ]:
data["tagged_lyric"] = tagged_lyric
data.head()

,id,year,title,singer,lyric,x_rated,cleaned_lyrics,tagged_lyric
0,5758967,1964,물새우는 해변,권혜경,고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 ...,False,고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 ...,"[고요한, 밤하늘, 별, 잠들다, 밀리다, 파도, 소리, 혼자, 들다, 외롭다, 홀..."
1,4083218,1964,내일또 만납시다,금호동,하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\...,False,하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\...,"[하루, 끝내다, 돌아가다, 거리, 물결, 하늘, 별, 하나, 둘, 반짝이다, 가로..."
2,3622464,1964,밀짚모자 목장아가씨,박재란,시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술...,False,시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술...,"[시원하다, 밀짚모자, 포플, 그늘, 양, 떼, 몰다, 가다, 목장, 아가씨, 연분..."
3,3621978,1964,아빠 안녕,현미,비가 나리네 비가 나리네\n돌아선 이가슴에\n그님은 어데 그어느곳에\n이마음 벗사려...,False,비가 나리네 비가 나리네\n돌아선 이가슴에\n그님은 어데 그어느곳에\n이마음 벗사려...,"[비, 나리, 비, 나리, 돌아서다, 가슴, 님, 어데, 어느, 곳, 마음, 벗, ..."
4,2517558,1964,빗속의 여인 1964년작,Add 4 신중현,잊지 못할 빗속의 여인\n지금은 어데 있나\n\n노오란 레인코트에\n검은 눈동자 잊...,False,잊지 못할 빗속의 여인\n지금은 어데 있나\n\n노오란 레인코트에\n검은 눈동자 잊...,"[잊다, 하다, 빗속, 여인, 지금, 어데, 있다, 놓다, 레인코트, 검다, 눈동자..."


- 결과 저장

In [ ]:
import pickle

# 파일객체를 생성 하고나면 반드시 해당 객체를 close() 해주어야 함
# with : 자동으로 해당 객체를 close() 해줌
# w : write, b : binary

with open("./data/tagged_lyric.pkl", "wb") as f :
  pickle.dump(data, f)

- 토큰화 결과 불러오기

In [ ]:
import pickle

# r : read

with open("./data/tagged_lyric.pkl", "rb") as f :
  data = pickle.load(f)

data.head()

,id,year,title,singer,lyric,x_rated,cleaned_lyrics,tagged_lyric
0,5758967,1964,물새우는 해변,권혜경,고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 ...,False,고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 ...,"[고요한, 밤하늘, 별, 잠들다, 밀리다, 파도, 소리, 혼자, 들다, 외롭다, 홀..."
1,4083218,1964,내일또 만납시다,금호동,하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\...,False,하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\...,"[하루, 끝내다, 돌아가다, 거리, 물결, 하늘, 별, 하나, 둘, 반짝이다, 가로..."
2,3622464,1964,밀짚모자 목장아가씨,박재란,시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술...,False,시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술...,"[시원하다, 밀짚모자, 포플, 그늘, 양, 떼, 몰다, 가다, 목장, 아가씨, 연분..."
3,3621978,1964,아빠 안녕,현미,비가 나리네 비가 나리네\n돌아선 이가슴에\n그님은 어데 그어느곳에\n이마음 벗사려...,False,비가 나리네 비가 나리네\n돌아선 이가슴에\n그님은 어데 그어느곳에\n이마음 벗사려...,"[비, 나리, 비, 나리, 돌아서다, 가슴, 님, 어데, 어느, 곳, 마음, 벗, ..."
4,2517558,1964,빗속의 여인 1964년작,Add 4 신중현,잊지 못할 빗속의 여인\n지금은 어데 있나\n\n노오란 레인코트에\n검은 눈동자 잊...,False,잊지 못할 빗속의 여인\n지금은 어데 있나\n\n노오란 레인코트에\n검은 눈동자 잊...,"[잊다, 하다, 빗속, 여인, 지금, 어데, 있다, 놓다, 레인코트, 검다, 눈동자..."


# 임베딩
- Doc2Vec을 이용
- Doc2Vec은 문서 하나를 하나의 vector화 (2015년 제안)
  - 주요 모델 : PV-DM model, PV-DBoW

- DM (Distributed Memory)
  - 벡터와 앞의 단어들을 사용해서 다음에 나오는 단어를 유추
  - 윈도우 크기 내의 단어들을 input으로 사용
  - 맨 앞에서부터 한 단어씩 훈련 데이터로 사용
  - 하나의 중심 단어를 output으로 학습 시키는 모델
  - 벡터가 학습 시 문서의 주제를 잡아주는 메모리와 같은 역할을 수행
  - 일반적으로 DBoW보다 더 성능이 우수
- DBoW (Distributed Bag Of Word)
  - ID를 가지고 단어를 랜덤하게 예측하는 방식을 사용

<center>  
<img src="https://arome1004.cafe24.com/images/machine_learning/doc2vec.png" width=50%>
</center>  

- Doc2Vec 준비
  - Doc2Vec 모델로 문서 벡터를 만들어내기 위해서는, **각 문서를 구분할 수 있는 태그(문서 ID)**와 해당 문서를 구성하는 **단어(토큰) 목록**이 함께 필요
  - gensim의 TaggedDocument 클래스 활용

In [ ]:
!pip install -q gensim

# 세션 다시 시작

- 문서 태깅 실습

In [ ]:
import gensim
from gensim.models.doc2vec import TaggedDocument

TaggedDocument(

    # 붙일 태그

    tags = ["document 0"],

    # 태그를 붙일 문서

    words = data["tagged_lyric"][0]

)

TaggedDocument(words=['고요한', '밤하늘', '별', '잠들다', '밀리다', '파도', '소리', '혼자', '들다', '외롭다', '홀로', '날다', '물새', '울음소리', '쓸쓸하다', '내', '마음', '속', '슬픔', '네', '고요한', '밤하늘', '별', '잠들다', '밀리다', '파도', '소리', '혼자', '들다', '외롭다', '홀로', '날다', '물새', '울음소리', '쓸쓸하다', '내', '마음', '속', '슬픔', '주네'], tags=['document 0'])

In [ ]:
# 태깅 결과 저장

tagged_corpus_list = []

# enumerate() : 값에서 0부터 시작하는 인덱스를 붙여줌

for i, tokens in enumerate(data["tagged_lyric"]) :

  # 문서에 붙일 태그명

  tag = f"document_{i}"

  # 각 가사 문서에 태그를 붙여서 리스트에 저장

  tagged_corpus_list.append(TaggedDocument(tags = [tag], words = tokens))



- 문서 벡터 생성을 위한 Doc2vec 학습

In [ ]:
from gensim.models import doc2vec

# Doc2vec 모델 초기화

model = doc2vec.Doc2Vec(

    vector_size = 300,  # 하나의 토큰과 연관된 토큰의 수
    alpha = 0.025,      # 초기 학습률
    min_alpha = 0.001,  # 반복시마다 감소되는 학습률 값
    window = 8,         # 분석에 사용할 토큰 주변의 토큰 수 (앞뒤 8개 토큰)
    min_count = 2,      # 빈도수가 2이하인 토큰은 제외
    dm = 1,             # 사용할 방법 : 0 (DBow), 1 (PV-DM)
    seed = 10
)

In [ ]:
# 단어 사전 생성

model.build_vocab(tagged_corpus_list)

In [ ]:
# 임베딩 수행

model.train(tagged_corpus_list,
            total_examples = model.corpus_count, # 전체 문서의 갯수
            epochs = 5)                          # epochs가 너무 크면 과적합 가능성이 높아짐

- 문서 벡터 값을 컬럼에 추가

In [ ]:
vector_list = [model.dv[f"document_{i}"] for i in range(len(data))]

- doc2vec_vector 컬럼에 결과 저장

In [ ]:
data["doc2vec_vector"] = vector_list
data.head()

,id,year,title,singer,lyric,x_rated,cleaned_lyrics,tagged_lyric,doc2vec_vector
0,5758967,1964,물새우는 해변,권혜경,고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 ...,False,고요한 밤하늘에 별이 잠들고\n밀리는 파도소리 나혼자 들으며\n외로히 홀로 날으는 ...,"[고요한, 밤하늘, 별, 잠들다, 밀리다, 파도, 소리, 혼자, 들다, 외롭다, 홀...","[-0.013662812, -0.0209884, 0.0077657937, -0.02..."
1,4083218,1964,내일또 만납시다,금호동,하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\...,False,하루의 일을 끝내고 돌아가는\n거리엔 사람의 물결\n하늘엔 별이 하나 둘 반짝이면\...,"[하루, 끝내다, 돌아가다, 거리, 물결, 하늘, 별, 하나, 둘, 반짝이다, 가로...","[0.022289274, -0.008954078, -0.018778576, -0.0..."
2,3622464,1964,밀짚모자 목장아가씨,박재란,시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술...,False,시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술...,"[시원하다, 밀짚모자, 포플, 그늘, 양, 떼, 몰다, 가다, 목장, 아가씨, 연분...","[0.01961494, -0.013863058, -0.036556512, -0.00..."
3,3621978,1964,아빠 안녕,현미,비가 나리네 비가 나리네\n돌아선 이가슴에\n그님은 어데 그어느곳에\n이마음 벗사려...,False,비가 나리네 비가 나리네\n돌아선 이가슴에\n그님은 어데 그어느곳에\n이마음 벗사려...,"[비, 나리, 비, 나리, 돌아서다, 가슴, 님, 어데, 어느, 곳, 마음, 벗, ...","[0.012313566, -0.0068599423, 0.014181502, -0.0..."
4,2517558,1964,빗속의 여인 1964년작,Add 4 신중현,잊지 못할 빗속의 여인\n지금은 어데 있나\n\n노오란 레인코트에\n검은 눈동자 잊...,False,잊지 못할 빗속의 여인\n지금은 어데 있나\n\n노오란 레인코트에\n검은 눈동자 잊...,"[잊다, 하다, 빗속, 여인, 지금, 어데, 있다, 놓다, 레인코트, 검다, 눈동자...","[0.021802993, -0.02055871, -0.031454574, 0.027..."


# 유사도 기반 추천 기능 구현

- 가사 기반의 코사인 유사도 계산

In [ ]:
# 1. 대상 곡의 벡터 추출


In [ ]:
# 2. 전체 곡 벡터를 하나의 배열로 생성


In [ ]:
# 3. 대상 곡(단일)과 전체 곡(다중) 간의 코사인 유사도 계산


In [ ]:
# 4. 자기 자신(대상 곡)은 추천 결과에서 제외하기 위해 유사도 값을 -1로 설정


In [ ]:
# 5. 유사도가 높은 상위 10개 곡의 인덱스 추출


In [ ]:
# 6. 최종 추천 결과 구성 (곡 id, title, singer, year 및 유사도 점수)


- 가사 기반 추천함수 구현

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommend_songs_by_lyric(doc_index, df, top_n = 10) :
    """
    doc_index : 추천을 원하는 곡의 데이터프레임 인덱스
    df        : 전체 곡 정보가 들어있는 데이터프레임
    top_n     : 상위 몇 개를 추천할지 (기본 10개)
    """

    # 1. 대상 곡의 벡터 추출
    # 코사인 유사도 함수가 2차원 데이터를 입력받음

    target_vector = df.loc[doc_index, "doc2vec_vector"].reshape(1, -1)

    # 2. 전체 곡 벡터를 하나의 배열로 생성

    all_vectors = np.stack(df["doc2vec_vector"].values, axis = 0)

    # 3. 대상 곡과 전체 곡 간의 코사인 유사도 계산 + 1차원 변환
    # flatten() : 결과 값이 2차원 데이터이므로 1차원으로 다시 변환 (정렬)

    simlarity_score = cosine_similarity(target_vector, all_vectors).flatten()

    # 4. 자기 자신(대상 곡)은 추천 결과에서 제외하기 위해 유사도 값을 -1로 설정
    # 오름차순 정렬하기 때문에 -1을 넣어주면 가장 마지막에 위치

    simlarity_score[doc_index] = -1

    # 5. 유사도가 높은 상위 10개 곡의 인덱스 추출
    # argsort() : 인덱스 오름차순 정렬
    # [::-1] : reverse (데이터의 순서를 반대로 배치) -> 내림차순

    top_indices = simlarity_score.argsort()[::-1][:top_n]

    # 6. 최종 추천 결과 구성 (곡 id, title, singer, year 및 유사도 점수)
    # 대상곡 정보

    target_df = df.loc[[doc_index], ["title", "singer", "year", "lyric"]].copy()

    # 추천곡 정보

    result_df = df.loc[top_indices, ["title", "singer", "year", "lyric"]].copy()

    # 유사도 점수 추가

    result_df["similarity_score"] = simlarity_score[top_indices]

    # 7. 최종 결과 반환

    return (target_df, result_df)

- 음악 추천하기 테스트

In [ ]:
idx = 100

target_info, recommand_info = recommend_songs_by_lyric(idx, data, top_n = 10)

display(target_info)
display(recommand_info)

,title,singer,year,lyric
100,서울이여 안녕,이미자,1968,안녕 안녕 서울이여 안녕\n그리운 님찾아 바다 건너 천리길\n쌓이고 쌓인 회포 풀려...


,title,singer,year,lyric,similarity_score
719,나도야 간다,김수철,1985,봄이 오는 캠퍼스\n잔디밭에\n팔베개를 하고 누워\n편지를 쓰네\n노랑나비 한마리\...,0.901210
541,청춘,산울림,1981,언젠간 가겠지 푸르른 이 청춘\n\n지고 또 피는 꽃잎처럼\n\n달 밝은 밤이면 창...,0.888654
470,여름,징검다리,1979,흥에 겨워 여름이 오면\n가슴을 활짝 열어요\n넝쿨장미 그늘 속에도\n젊음이 넘쳐 ...,0.888289
636,못다핀 꽃 한 송이,김수철,1984,언제 가셨는데 안오시나\n한잎두고 가신님아\n가지위에 눈물 적셔놓고\n이는 바람소리...,0.884459
803,새벽기차,다섯손가락,1986,해지고 어두운 거리를\n나 홀로 걸어가면은\n눈물처럼 젖어드는 슬픈 이별이\n떠나간...,0.879798
134,천리길,나훈아,1969,돌뿌리 가시밭 길\n산을 넘어 천리길\n반겨주실 님을 찾아\n강을 건너 천리길\n아...,0.878341
295,고향초,홍민,1974,남쪽 나라 바다 멀리\n물새가 날으면\n뒷동산에 동백꽃도 곱게 피는데\n뽕을 따던 ...,0.872436
1174,언제나 타인,김수철,1990,만남에서 몰래 빠져나와\n너와나 이별도 없이\n그냥 멀어져 버린\n우리는 언제나 타...,0.871597
465,하얀 면사포,어니언스 이수영,1979,창밖에 낙엽지고 그대 떠나가면\n허전한 내마음은 달랠 길 없다오\n웃으며 떠나야 할...,0.871482
3819,소나기,아이오아이 IOI,2017,이 비가\n머리 위로 쏟아지면\n흠뻑 젖고 말겠죠\n내 마음도\n머물러줘요\n아직까...,0.865201


- 메타데이터를 활용한 추천 성능 개선
  - 가사 외에도 연도나 가수 정보를 추가하여 성능 개선

  - 이 외에도 아래 방법들로 성능 개선 가능
    - 최신 딥러닝 임베딩 활용 문서의 뉘앙스를 보다 세밀하게 포착 가능
    - 하이브리드 필터링 방법 채택 콘텐츠 기반과 협업 필터링 방식을 결합하여 사용자 행동 데이터 반영
    - 모델 및 파라미터 최적화
      - Doc2Vec 외 TF-IDF, LDA 등 다른 벡터화 기법과 앙상블 적용
      - 학습 epochs, vector_size, window 등 하이퍼파라미터 튜닝
    - 전처리 및 토큰화, 정규화, 노이즈 제거, 중복 제거 등 데이터 클린징 강화

- 가사 유사도 계산

In [ ]:
# 대상 곡의 Doc2Vec 벡터 추출 및 2D 배열로 변환

idx = 100
target_vector = data.loc[idx, "doc2vec_vector"].reshape(1, -1)

target_vector.shape

(1, 300)

In [ ]:
# 전체 곡의 Doc2Vec 벡터를 하나의 2D 배열로 결합

all_vector = np.stack(data["doc2vec_vector"].values, axis = 0)

all_vector.shape

(4458, 300)

In [ ]:
# 대상 곡과 전체 곡 간의 코사인 유사도 계산 후 1D 배열로 변환



- 연도 유사도 계산

In [ ]:
# 대상 곡의 연도 추출


In [ ]:
# 전체 곡의 연도 배열 생성 (스케일러 활용을 위한 1차원 배열 → 2차원 배열로 변환 필요)


In [ ]:
# 대상 곡과 각 곡 간의 연도 차이의 절대값 계산


In [ ]:
# 대상 곡과 다른 곡들 간의 연도 차이(year_diff)를 0~1 범위로 정규화


In [ ]:
# 낮은 차이가 높은 유사도가 되도록 1에서 빼줌


In [ ]:
# 상위 10개 연도 유사도 값 확인


- 가수 유사도 계산

In [ ]:
# 대상 곡의 가수 추출


In [ ]:
# 전체 곡의 가수와 대상 곡의 가수 비교하여 동일하면 1, 아니면 0


- 가중치를 적용하여 최종 유사도 계산

In [ ]:
# 각 요소에 부여할 가중치 설정


In [ ]:
# 각 유사도 값에 가중치를 곱한 후 합산하여 최종 유사도 계산


- 자기 자신은 추천에서 제외

- 최종 유사도가 높은 상위 top_n 곡 인덱스 추출

- 추천 함수 정의
  - 가사, 년도, 가수 정보를 포함하여 추천

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def recommend_songs_by_lyric_metadata(doc_index, df,
                                      top_n = 10,
                                      lyric_weight = 1.0,
                                      year_weight = 0.5,
                                      singer_weight = 0.2) :
    """
    메타데이터(가사, 연도, 가수)를 활용한 추천 시스템.
    주어진 곡(doc_index)을 기준으로 가사, 연도, 가수 유사도를 계산한 후,
    각 유사도에 가중치를 곱해 최종 유사도를 산출하고, 상위 top_n곡을 추천한다.

    매개변수:
    - doc_index   : 추천 기준이 되는 곡의 DataFrame 인덱스
    - df          : 전체 곡 정보가 담긴 DataFrame (doc2vec_vector, year, singer 컬럼 포함)
    - top_n       : 추천할 곡의 개수 (기본 10개)
    - lyric_weight: 가사 유사도에 부여할 가중치 (기본 1.0)
    - year_weight : 연도 유사도에 부여할 가중치 (기본 0.5)
    - singer_weight: 가수 유사도에 부여할 가중치 (기본 0.2)

    반환:
    - target_df : 기준 곡의 정보 (title, singer, year, lyric)
    - rec_df    : 추천된 곡들의 정보 및 최종 유사도(final_similarity)를 포함한 DataFrame
    """
    # 1. 가사 유사도 계산 (Doc2Vec 기반 코사인 유사도)

    target_vector = data.loc[doc_index, "doc2vec_vector"].reshape(1, -1)
    all_vectors = np.stack(data["doc2vec_vector"].values, axis = 0)
    similarity_score = cosine_similarity(target_vector, all_vectors).flatten()

    # 2. 연도 유사도 계산 (년도의 차이를 계산해서 스케일링)
    # 대상 노래의 년도

    target_year = data.loc[doc_index, "year"]

    # 전체 노래의 년도

    all_years = df["year"].values.reshape(1, -1)

    # 대상 년도와 전체 년도간의 차이를 계산

    year_diff = np.abs(all_years - target_year)

    # 스케일링(0-1)

    scaler = MinMaxScaler()
    year_diff_scaled = scaler.fit_transform(year_diff)

    # 낮은 차이가 높은 유사도가 되도록 1에서 빼줌

    year_sim = 1 - year_diff_scaled.flatten()

    # 3. 가수 유사도 계산
    # 동일한 가수이면 1, 그렇지 않으면 0

    target_singer = df.loc[doc_index, "singer"]

    # df["singer"] 컬럼의 값을 하나씩 읽어서 singer에 저장
    # singer와 target_singer 같으면 리스트에 1을 저장하고 그렇지 않으면 0을 저장

    singer_sim = np.array([1 if singer == target_singer else 0 for singer in df["singer"].values])

    # 4. 가중치 적용하여 최종 유사도 계산

    final_sim = (lyric_weight * similarity_score) + (year_weight * year_sim) + (singer_weight * singer_sim)

    # 5. 대상 곡은 자기 자신과의 유사도가 항상 높게 나오므로 추천 결과에서 제외

    final_sim[doc_index] = -np.inf

    # 6. 최종 유사도가 높은 상위 top_n 곡의 인덱스 추출

    top_indices = np.argsort(final_sim)[::-1][:top_n]

    # 7. 결과 DataFrame 구성: 기준 곡 정보와 추천된 곡 정보(최종 유사도 포함)

    target_df = df.loc[[doc_index], ["title", "singer", "year", "lyric"]].copy()
    rec_df = df.loc[top_indices, ["title", "singer", "year", "lyric"]].copy()
    rec_df["final_similarity"] = final_sim[top_indices]

    # 8. 결과값 리턴

    return (target_df, rec_df)


In [ ]:
idx = 20

target_info, recommand_info = recommend_songs_by_lyric_metadata(idx, data, top_n = 10)

display(target_info)
display(recommand_info)

,title,singer,year,lyric
20,진주조개잡이,박재란,1965,새파란 수평선 흰구름\n흐르는 오늘도 즐거워라\n조개잡이 하는 처녀들\n흥겨운 젊은...


,title,singer,year,lyric,final_similarity
2,밀짚모자 목장아가씨,박재란,1964,시원한 밀짚모자\n포플라 그늘에\n양떼를 몰고가는\n목장의 아가씨\n연분홍 빛 입술...,1.597088
10,소쩍새 우는 마을,박재란,1964,소쩍 소쩍새 울고간 뒤에 나풀나풀 나비가 춤추며 오네\n두꺼비도 잠 깨어 하품하는데...,1.513481
1611,하루,장필순,1995,도시의 희뿌연 아침 열리고\n가로수 긴 팔 벌려 하품할 때\n그대의 머리 위에\n야...,1.442212
300,하얀 조가비,박인희,1974,고동을 불어본다\n하얀 조가비\n먼 바닷 물소리가\n다시 그리워\n노을진 수평선에\...,1.431606
557,바윗돌,정오차,1982,찬비 맞으며\n눈물만 흘리고\n하얀 눈 맞으며\n아픔만 달래는\n바윗돌\n세상만사 ...,1.411344
2576,광화문 연가,이수영,2004,이제모두 세월따라 흔적도 없이 변해갔지만\n덕수궁 돌담길엔 아직 남아 있어요\n다정...,1.410179
4436,봄여름가을겨울 Still Life,BIGBANG 빅뱅,2023,이듬해 질 녘 꽃 피는 봄 한여름 밤의 꿈\n가을 타 겨울 내릴 눈 1년 네 번 또...,1.408860
3725,Rain,태연 TAEYEON,2016,텅 빈 회색 빛 거린 참 허전해\n쓸쓸한 기분에 유리창을 열어\n내민 두 손 위로 ...,1.407306
21,산넘어 남촌에는,박재란,1965,산너머 남촌에는 누가 살길래\n해마다 봄바람이 남으로 오네\n아꽃피는 사월이면 진달...,1.405917
1829,사랑이라는 이유로,조 트리오,1997,사랑이라는 이유로\n하얗게 새운 많은 밤들\n이젠 멀어져 기억 속으로 묻혀\n함께 ...,1.404745


# 콘텐츠 기반 필터링 추천시스템 정리

### 주요 특징
- <span style="color:red"><u><b>아이템 특성 활용:</b></u></span> 아이템(예: 가사, 장르, 연도, 가수 등)의 내용이나 메타데이터를 활용  
- <span style="color:red"><u><b>개별 아이템 분석:</b></u></span> 사용자가 평가한 아이템과 **유사한 속성**을 가진 다른 아이템 추천  
- <span style="color:red"><u><b>설명 가능성:</b></u></span> 추천 이유를 아이템의 특성으로 **명확하게 설명** 가능
- <u><b>초기 콜드 스타트</b></u> 문제에 대응 가능
    - Cold start : 서비스에 사용자나 아이템에 관한 정보가 적은 경우, 즉 <u><b>신규 사용자 혹은 신규 아이템을 추천하기 어려운 문제</b></u>
    - <u><b>협업 필터링</b></u>은 과거 사용자의 선호 데이터가 필요하므로 <u><b>콜드 스타트 문제에 취약함</b></u>
    - 콜드 스타트 문제와 성능 향상을 위해 <u><b>신규 고객에게는 콘텐츠 기반 필터링</b></u>, <u><b>특정 횟수 이상 구매 회원에게는 협업 필터링</b></u>을 적용하는 방식도 존재함

> 과도한 특수화 (Over Specialization) 문제 존재
> - 추천 상품의 다양성을 보장할 수 없는 문제

### 장점과 단점

| **구분** | **세부 내용** |
|----------|---------------|
| **장점** | - 설명 가능하고 직관적임<br>- 사용자의 개별 취향 반영 가능<br>- 사용자 데이터가 상대적으로 적어도 적용 가능 |
| **단점** | - 아이템의 특징에 한정되어 다양성이 부족할 수 있음<br>- <span style="color:red"><u><b>과도한 특수화 (Over Specialization)</b></u></span> 문제<br>- 콘텐츠 특징 추출 및 전처리 비용 발생<br>- 새로운 아이템의 경우 충분한 특징 추출이 어려움 (<span style="color:red"><u><b>Cold Start 문제</b></u></span>)<br>- 사용자가 기존에 소비한 아이템과 너무 유사한 추천 제공<br>- 콘텐츠 정보 부족 시 추천 성능 저하 |


### 콘텐츠 기반 필터링의 한계와 협업 필터링으로의 전환

> **한계 극복:**  
> - 콘텐츠 기반 필터링은 **아이템 내재적 정보**에 의존하므로, 사용자의 **잠재적 취향**이나 **다른 사용자와의 상호작용 정보**를 반영하기 어려움

> **협업 필터링 소개:**  
> - <span style="color:red"><u><b>사용자 행동 기반:</b></u></span> 여러 사용자의 평가나 소비 데이터를 활용하여 **사용자 간 혹은 아이템 간 유사도**를 산출  
> - <span style="color:red"><u><b>다양한 추천:</b></u></span> 콘텐츠 기반의 한계를 보완해 사용자의 **다양한 취향**과 **새로운 아이템**을 추천 가능  
> - <span style="color:red"><u><b>하이브리드 접근:</b></u></span> 콘텐츠 기반과 협업 필터링을 결합하여 **더욱 정교한 추천 시스템** 구현